## 3D image classification from CT scans

### Setup

In [1]:
import os
import zipfile
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from scipy import ndimage
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
from torchvision import datasets,transforms
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Downloading the MosMedData: Chest CT Scans with COVID-19 Related Findings

In [ ]:
!wget https://github.com/hasibzunair/3D-image-classification-tutorial/releases/download/v0.2/CT-0.zip 

In [ ]:
!wget https://github.com/hasibzunair/3D-image-classification-tutorial/releases/download/v0.2/CT-23.zip

In [14]:
os.makedirs("MosMedData")

with zipfile.ZipFile("CT-0.zip", "r") as z_fp:
    z_fp.extractall("./MosMedData/")

with zipfile.ZipFile("CT-23.zip", "r") as z_fp:
    z_fp.extractall("./MosMedData/")

#### Loading data and preprocessing

### Build train and validation datasets

In [3]:
class MosMedDataDataset(Dataset):
    def __init__(self, root, transform=None, target_transform=None, train=True):
        self.root = root
        self.transform = transform
        self.target_transform = target_transform
        self.train = train

        normal_scan_paths = glob.glob(os.path.join(self.root, "CT-0/*.nii.gz"))
        abnormal_scan_paths = glob.glob(os.path.join(self.root, "CT-23/*.nii.gz"))

        normal_labels = torch.tensor([0 for _ in range(len(normal_scan_paths))])
        abnormal_labels = torch.tensor([1 for _ in range(len(abnormal_scan_paths))])

        if train:
            self.files = abnormal_scan_paths[:70] + normal_scan_paths[:70]
            self.labels = torch.concat((abnormal_labels[:70], normal_labels[:70]), dim=0)
        else:
            self.files = abnormal_scan_paths[70:] + normal_scan_paths[70:]
            self.labels = torch.concat((abnormal_labels[70:], normal_labels[70:]), dim=0)
        
        self.shuffle()


    def shuffle(self):
        indices = list(range(len(self.files)))
        random.seed(42)
        random.shuffle(indices)
        self.files = [self.files[i] for i in indices]
        self.labels = [self.labels[i] for i in indices]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        image_path = self.files[index]
        image = nib.load(image_path).get_fdata()
        label = self.labels[index]

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [7]:
IMG_SIZE = (64, 128, 128)
BATCH_SIZE = 32
data_path = "MosMedData"

train_dataset = MosMedDataDataset(data_path, transform=transforms.ToTensor(), train=True)
test_dataset = MosMedDataDataset(data_path, transform=transforms.ToTensor(), train=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


### Data augmentation

#### Define a 3D convolutional neural network

In [3]:
class CNN3D(nn.Module):
    def __init__(self, in_channels: int, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.layers = nn.ModuleList()

        channels = [in_channels, 64, 64, 128, 256]
        for index, size in enumerate(channels[:-1]):
            self.layers.append(nn.Sequential(
                nn.Conv3d(in_channels=size,
                          out_channels=channels[index+1], kernel_size=3),
                nn.ReLU(),
                nn.MaxPool3d(kernel_size=2),
                nn.BatchNorm3d(channels[index+1])))

        self.layers.append(nn.Sequential(
            nn.AdaptiveAvgPool3d(1),
            nn.Flatten(),
            nn.Linear(in_features=256, out_features=512),
            nn.ReLU(),
            nn.Dropout1d(p=0.3),
            nn.Linear(in_features=512, out_features=1),
            nn.Sigmoid()))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for layer in self.layers:
            x = layer(x)
        return x

In [4]:
model = CNN3D(1)

In [5]:
summary(model, input_size=(1, 64, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1     [-1, 64, 62, 126, 126]           1,792
              ReLU-2     [-1, 64, 62, 126, 126]               0
         MaxPool3d-3       [-1, 64, 31, 63, 63]               0
       BatchNorm3d-4       [-1, 64, 31, 63, 63]             128
            Conv3d-5       [-1, 64, 29, 61, 61]         110,656
              ReLU-6       [-1, 64, 29, 61, 61]               0
         MaxPool3d-7       [-1, 64, 14, 30, 30]               0
       BatchNorm3d-8       [-1, 64, 14, 30, 30]             128
            Conv3d-9      [-1, 128, 12, 28, 28]         221,312
             ReLU-10      [-1, 128, 12, 28, 28]               0
        MaxPool3d-11       [-1, 128, 6, 14, 14]               0
      BatchNorm3d-12       [-1, 128, 6, 14, 14]             256
           Conv3d-13       [-1, 256, 4, 12, 12]         884,992
             ReLU-14       [-1, 256, 4,

### Train model

### Visualizing model performance

### Make predictions on a single CT scan